 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Fully-connected-linear-network" data-toc-modified-id="Fully-connected-linear-network-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Fully connected linear network</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Get-temperature-data" data-toc-modified-id="Get-temperature-data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Get temperature data</a></span></li><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Build-fully-connected-model" data-toc-modified-id="Build-fully-connected-model-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Build fully connected model</a></span></li><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Predict-for-one-day" data-toc-modified-id="Predict-for-one-day-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Predict for one day</a></span></li><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Post-processing-with-rolling-window-for-2016" data-toc-modified-id="Post-processing-with-rolling-window-for-2016-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Post processing with rolling window for 2016</a></span></li><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Train-2015,-predict-2016" data-toc-modified-id="Train-2015,-predict-2016-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Train 2015, predict 2016</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Neural-network-with-one-hidden-layer" data-toc-modified-id="Neural-network-with-one-hidden-layer-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Neural network with one hidden layer</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Build-network" data-toc-modified-id="Build-network-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Build network</a></span></li><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Train-2015,-predict-2016" data-toc-modified-id="Train-2015,-predict-2016-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Train 2015, predict 2016</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Hidden-model-with-station-embeddings" data-toc-modified-id="Hidden-model-with-station-embeddings-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Hidden model with station embeddings</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Get-the-station-data-and-prepare-as-input-for-embedding-layer" data-toc-modified-id="Get-the-station-data-and-prepare-as-input-for-embedding-layer-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Get the station data and prepare as input for embedding layer</a></span></li><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Build-embedding-model" data-toc-modified-id="Build-embedding-model-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Build embedding model</a></span></li><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Train-2015,-predict-2016" data-toc-modified-id="Train-2015,-predict-2016-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Train 2015, predict 2016</a></span></li><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Embedding-size-hyper-parameter-tuning" data-toc-modified-id="Embedding-size-hyper-parameter-tuning-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Embedding size hyper-parameter tuning</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Adding-additional-variables-to-hidden-layer-model" data-toc-modified-id="Adding-additional-variables-to-hidden-layer-model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Adding additional variables to hidden layer model</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Get-additional-variables" data-toc-modified-id="Get-additional-variables-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Get additional variables</a></span></li><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Build-model-and-predict-for-2016" data-toc-modified-id="Build-model-and-predict-for-2016-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Build model and predict for 2016</a></span></li><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Hidden-nodes-hyper-parameter-tuning" data-toc-modified-id="Hidden-nodes-hyper-parameter-tuning-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Hidden nodes hyper-parameter tuning</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Additional-variables-with-the-embedding-model" data-toc-modified-id="Additional-variables-with-the-embedding-model-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Additional variables with the embedding model</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/fc_network.ipynb#Build-model-and-predict-for-2016" data-toc-modified-id="Build-model-and-predict-for-2016-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Build model and predict for 2016</a></span></li></ul></li></ul></div>

# Fully connected neural networks

In this notebook we will expand the simple EMOS linear network to fully connected non-linear neural nets. Furthermore, we will also use auxiliary data.

In [84]:
# Imports
from importlib import reload
import emos_network_theano; reload(emos_network_theano)
from  emos_network_theano import EMOS_Network
from crps_loss import crps_cost_function
import utils; reload(utils)
from utils import *
from datetime import datetime
%matplotlib inline
import matplotlib.pyplot as plt
from collections import OrderedDict
# import the keras modules
# Note that the cost function only works with the theano backend
import keras
from keras.layers import Input, Dense, merge, Embedding, Flatten, Dropout
from keras.layers.merge import Concatenate
from keras.models import Model
import keras.backend as K
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD, Adam

In [3]:
# Basic setup
DATA_DIR = '/Volumes/STICK/data/ppnn_data/'  # Mac
# DATA_DIR = '/project/meteo/w2w/C7/ppnn_data/'   # LMU
fn = 'data_interpolated_00UTC.nc'   # Temperature observation and forecast data
window_size = 25   # Days in rolling window
fclt = 48   # Forecast lead time in hours

## Fully connected linear network

As a first step, we can build a linear model which also connects the means and standard deviations.

### Get temperature data

This follows the steps in the EMOS Network data

In [4]:
# Load the full dataset
tobs_full, tfc_full, dates = load_nc_data(DATA_DIR + fn)
tobs_full.shape, tfc_full.shape, dates.shape
# Pick an example day
date_idx = return_date_idx(dates, 2011, 2, 14)   # year, month, day
# Now get the training and test data for this day
tfc_mean_train, tfc_std_train, tobs_train, tfc_mean_test, tfc_std_test, tobs_test = \
        get_train_test_data(tobs_full, tfc_full, date_idx, window_size, fclt, 
                            subtract_std_mean=False)

### Build fully connected model

In [5]:
def build_fc_model():
    inp = Input(shape=(2,))
    x = Dense(2, activation='linear')(inp)
    return Model(inputs=inp, outputs=x)

In [18]:
fc_model = build_fc_model()
fc_model.compile(optimizer=SGD(0.1), loss=crps_cost_function)

In [7]:
fc_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 6         
Total params: 6
Trainable params: 6
Non-trainable params: 0
_________________________________________________________________


Now we have 6 parameters instead of 4 with the standard EMOS Network.

### Predict for one day

In [12]:
# Define some parameters
early_stopping_delta = 1e-4   # How much the CRPS must improve before stopping
steps_max = 1000   # How many steps to fit at max
batch_size = tfc_mean_train.shape[0]

In [13]:
# We also need to concatenate the input arrays
in_train = np.column_stack([tfc_mean_train, tfc_std_train])
in_test = np.column_stack([tfc_mean_test, tfc_std_test])
in_train.shape

(12619, 2)

In [19]:
fc_model.fit(in_train, tobs_train, epochs=steps_max, batch_size=batch_size,
          validation_data=[in_test, tobs_test], verbose=0,
          callbacks=[EarlyStopping(monitor='loss', min_delta=early_stopping_delta,
                                  patience=2)])

In [21]:
# Get train and test CRPS
(fc_model.evaluate(in_train, tobs_train, batch_size, verbose=0), 
 fc_model.evaluate(in_test, tobs_test, batch_size, verbose=0))

(1.224784107227332, 0.75179883171599682)

For this particular day we get a score that is slightly better than the standard EMOS network.

### Post processing with rolling window for 2016

As with the EMOS models let's do a rolling window global post-processing for 2016.

In [22]:
date_idx_start = return_date_idx(dates, 2016, 1, 1)
date_idx_stop = return_date_idx(dates, 2016, 12, 31)

In [26]:
fc_model = build_fc_model()
fc_model.compile(optimizer=SGD(0.1), loss=crps_cost_function, 
                    metrics=[crps_cost_function])
# Note that we have to define a metric in the corrent loop implementation

In [27]:
# Use the loop function in utils
train_crps_list, valid_crps_list = loop_over_days(
    fc_model,
    tobs_full, 
    tfc_full, 
    date_idx_start, date_idx_stop, 
    window_size=window_size,
    fclt=fclt,     
    epochs_max=steps_max, 
    early_stopping_delta=early_stopping_delta, 
    lr=0.1,   
    verbose=0)

3300
3400
3500
3600
Time: 196.83 s


In [28]:
np.mean(train_crps_list), np.mean(valid_crps_list)

(0.97781397659532787, 1.0036478569781389)

So we get a slightly better training score and a slightly worse test score. This is a sign of overfitting. But the differences are small.

### Train 2015, predict 2016

In [63]:
tfc_mean_train, tfc_std_train, tobs_train, tfc_mean_test, tfc_std_test, tobs_test = \
        get_train_test_data(tobs_full, tfc_full, date_idx_start, window_size=365, fclt=0, 
                            subtract_std_mean=False, test_plus=365)

In [64]:
fc_model = build_fc_model()
fc_model.compile(optimizer=SGD(0.1), loss=crps_cost_function)

In [65]:
in_train = np.column_stack([tfc_mean_train, tfc_std_train])
in_test = np.column_stack([tfc_mean_test, tfc_std_test])
in_train.shape

(180849, 2)

In [66]:
fc_model.fit(in_train, tobs_train, epochs=10, batch_size=1024,
             validation_data=[in_test, tobs_test])

Train on 180849 samples, validate on 181718 samples
Epoch 1/10
180849/180849 [==============================] - 0s - loss: 2.7792 - val_loss: 1.6748
Epoch 2/10
180849/180849 [==============================] - 0s - loss: 1.3894 - val_loss: 1.2322
Epoch 3/10
180849/180849 [==============================] - 0s - loss: 1.1722 - val_loss: 1.0815
Epoch 4/10
180849/180849 [==============================] - 0s - loss: 1.0971 - val_loss: 1.0317
Epoch 5/10
180849/180849 [==============================] - 0s - loss: 1.0762 - val_loss: 1.0172
Epoch 6/10
180849/180849 [==============================] - 0s - loss: 1.0709 - val_loss: 1.0140
Epoch 7/10
180849/180849 [==============================] - 0s - loss: 1.0696 - val_loss: 1.0122
Epoch 8/10
180849/180849 [==============================] - 0s - loss: 1.0692 - val_loss: 1.0124
Epoch 9/10
180849/180849 [==============================] - 0s - loss: 1.0692 - val_loss: 1.0115
Epoch 10/10
180849/180849 [==============================] - 0s - loss: 1.0

Very similar to the standard EMOS Network. This indicates that there is not much additional information in the two extra connections we added.

## Neural network with one hidden layer

Now we will build the first neural network with a hidden layer and a non-linear activation function. We will restrict ourselves to testing the 2015 training, 2016 prediction case.

### Build network

In [33]:
# Define parameters
hidden_nodes = 10

In [109]:
def build_hidden_model(hidden_nodes, feature_size=2):
    inp = Input(shape=(feature_size,))
    x = Dense(hidden_nodes, activation='relu')(inp)
    x = Dense(2, activation='linear')(x)
    return Model(inputs=inp, outputs=x)

In [35]:
hidden_model = build_hidden_model(hidden_nodes)
hidden_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                30        
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 22        
Total params: 52
Trainable params: 52
Non-trainable params: 0
_________________________________________________________________


In [41]:
hidden_model.compile(optimizer=Adam(0.01), loss=crps_cost_function)

### Train 2015, predict 2016

In [67]:
# We can use the same data from above!
hidden_model.fit(in_train, tobs_train, epochs=10, batch_size=4096,
                 validation_data=[in_test, tobs_test])

Train on 180849 samples, validate on 181718 samples
Epoch 1/10
180849/180849 [==============================] - 0s - loss: 1.0643 - val_loss: 1.0191
Epoch 2/10
180849/180849 [==============================] - 0s - loss: 1.0614 - val_loss: 1.0219
Epoch 3/10
180849/180849 [==============================] - 0s - loss: 1.0614 - val_loss: 1.0235
Epoch 4/10
180849/180849 [==============================] - 0s - loss: 1.0613 - val_loss: 1.0202
Epoch 5/10
180849/180849 [==============================] - 0s - loss: 1.0613 - val_loss: 1.0201
Epoch 6/10
180849/180849 [==============================] - 0s - loss: 1.0613 - val_loss: 1.0263
Epoch 7/10
180849/180849 [==============================] - 0s - loss: 1.0618 - val_loss: 1.0212ss: 1.
Epoch 8/10
180849/180849 [==============================] - 0s - loss: 1.0612 - val_loss: 1.0221
Epoch 9/10
180849/180849 [==============================] - 0s - loss: 1.0613 - val_loss: 1.0221
Epoch 10/10
180849/180849 [==============================] - 0s - los

The training loss is consistently smaller than the validation loss. This indicates overfitting. We could use Dropout for regularization. But since this model is just an intermediate step anyway, we will ignore that for now. The differences are small anyway.

## Hidden model with station embeddings

Next we will add a station embedding

### Get the station data and prepare as input for embedding layer
For this we will use a more general data loading function which will also be able to handle the auxiliary data. Eventually I could maybe write one general function. 

In [68]:
target, features, dates = prepare_data(DATA_DIR)

In [69]:
features.shape

(2, 3653, 537)

In [70]:
features_train, target_train, features_test, target_test, id_array_train, id_array_test = \
    scale_and_split(target, features, date_idx_start, 365, return_id_array=True)

In [71]:
features_train.shape, features_test.shape, id_array_train.shape

((180849, 2), (181718, 2), (180849,))

### Build embedding model

In [110]:
def build_emb_model(hidden_nodes, emb_size, max_id, feature_size=2):
    features_in = Input(shape=(feature_size,))
    id_in = Input(shape=(1,))
    emb = Embedding(max_id + 1, emb_size)(id_in)
    emb = Flatten()(emb)
    x = Concatenate()([features_in, emb])
    x = Dense(hidden_nodes, activation='relu')(x)
    x = Dense(2, activation='linear')(x)
    model = Model(inputs=[features_in, id_in], outputs=x)
    return model

In [79]:
emb_size = 5
max_id = int(np.max([id_array_test.max(), id_array_train.max()]))
hidden_nodes, max_id

(10, 536)

In [95]:
emb_model = build_emb_model(hidden_nodes, emb_size, max_id)
emb_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_19 (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_5 (Embedding)          (None, 1, 5)          2685        input_19[0][0]                   
____________________________________________________________________________________________________
input_18 (InputLayer)            (None, 2)             0                                            
____________________________________________________________________________________________________
flatten_5 (Flatten)              (None, 5)             0           embedding_5[0][0]                
___________________________________________________________________________________________

In [96]:
emb_model.compile(optimizer=Adam(0.01), loss=crps_cost_function)

### Train 2015, predict 2016

In [97]:
emb_model.fit([features_train, id_array_train], target_train, epochs=10, 
              batch_size=1024, 
              validation_data=[[features_test, id_array_test], target_test])

Train on 180849 samples, validate on 181718 samples
Epoch 1/10
180849/180849 [==============================] - 0s - loss: 3.1504 - val_loss: 0.9579
Epoch 2/10
180849/180849 [==============================] - 0s - loss: 0.9834 - val_loss: 0.9228
Epoch 3/10
180849/180849 [==============================] - 0s - loss: 0.9741 - val_loss: 0.9163
Epoch 4/10
180849/180849 [==============================] - 0s - loss: 0.9723 - val_loss: 0.9144
Epoch 5/10
180849/180849 [==============================] - 0s - loss: 0.9719 - val_loss: 0.9147
Epoch 6/10
180849/180849 [==============================] - 0s - loss: 0.9714 - val_loss: 0.9145
Epoch 7/10
180849/180849 [==============================] - 0s - loss: 0.9712 - val_loss: 0.9112
Epoch 8/10
180849/180849 [==============================] - 0s - loss: 0.9702 - val_loss: 0.9151
Epoch 9/10
180849/180849 [==============================] - 0s - loss: 0.9707 - val_loss: 0.9117
Epoch 10/10
180849/180849 [==============================] - 0s - loss: 0.9

### Embedding size hyper-parameter tuning

Since embeddings appear to work very well, we will test the impact of the embedding size before building more complex models. Of course, a larger embedding size might be useful when adding more variables, but this should give us some feeling.

In [106]:
def build_and_run_emb_model(emb_size):
    emb_model = build_emb_model(hidden_nodes, emb_size, max_id)
    emb_model.compile(optimizer=Adam(0.01), loss=crps_cost_function)
    emb_model.fit([features_train, id_array_train], target_train, epochs=20, 
                  batch_size=1024, verbose=0,
                  validation_data=[[features_test, id_array_test], target_test])
    print(emb_model.evaluate([features_train, id_array_train], target_train, verbose=0),
          emb_model.evaluate([features_test, id_array_test], target_test, verbose=0))

In [107]:
for emb_size in [1, 2, 3, 5, 10, 20]:
    print(emb_size)
    build_and_run_emb_model(emb_size)

1
0.967700981302 0.928655344943
2
0.960589448223 0.917231013011
3
0.96050738703 0.920315323502
5
0.958451787728 0.923359214098
10
0.960904902009 0.925523586535
20
0.959896959686 0.919102417839


Note that there is some variability. In our first experiment above with an embedding size of 5 we got a better score than here. For this very simple network an embedding size of two seems sufficient.

## Adding additional variables to hidden layer model

Now we can try adding additional variables.

### Get additional variables

Using the function defined in utils.

In [116]:
# The prepare_data function takes an ordered dict as an input
aux_dict = OrderedDict()
aux_dict['data_aux_geo_interpolated.nc'] = ['orog', 
                                            'station_alt', 
                                            'station_lat', 
                                            'station_lon']
aux_dict['data_aux_pl500_interpolated_00UTC.nc'] = ['u_pl500_fc',
                                                    'v_pl500_fc',
                                                    'gh_pl500_fc']
aux_dict['data_aux_pl850_interpolated_00UTC.nc'] = ['u_pl850_fc',
                                                    'v_pl850_fc',
                                                    'q_pl850_fc']
aux_dict['data_aux_surface_interpolated_00UTC.nc'] = ['cape_fc',
                                                      'sp_fc',
                                                      'tcc_fc']

In [117]:
target, features, dates = prepare_data(DATA_DIR, aux_dict)

In [118]:
features.shape

(24, 3653, 537)

In [119]:
features_train, target_train, features_test, target_test, id_array_train, id_array_test = \
    scale_and_split(target, features, date_idx_start, 365, return_id_array=True)

In [120]:
features_train.shape

(180849, 24)

### Build model and predict for 2016

In [124]:
hidden_model = build_hidden_model(hidden_nodes, feature_size=features_train.shape[1])
hidden_model.compile(optimizer=Adam(0.01), loss=crps_cost_function)

In [125]:
hidden_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_45 (InputLayer)        (None, 24)                0         
_________________________________________________________________
dense_45 (Dense)             (None, 10)                250       
_________________________________________________________________
dense_46 (Dense)             (None, 2)                 22        
Total params: 272
Trainable params: 272
Non-trainable params: 0
_________________________________________________________________


In [126]:
hidden_model.fit(features_train, target_train, epochs=10, batch_size=1024,
                 validation_data=[features_test, target_test])

Train on 180849 samples, validate on 181718 samples
Epoch 1/10
180849/180849 [==============================] - 0s - loss: 3.0880 - val_loss: 1.7341
Epoch 2/10
180849/180849 [==============================] - 0s - loss: 1.2352 - val_loss: 1.1541
Epoch 3/10
180849/180849 [==============================] - 0s - loss: 1.0377 - val_loss: 0.9793
Epoch 4/10
180849/180849 [==============================] - 0s - loss: 0.9775 - val_loss: 0.9483
Epoch 5/10
180849/180849 [==============================] - 0s - loss: 0.9706 - val_loss: 0.9398
Epoch 6/10
180849/180849 [==============================] - 0s - loss: 0.9677 - val_loss: 0.9378
Epoch 7/10
180849/180849 [==============================] - 0s - loss: 0.9673 - val_loss: 0.9415
Epoch 8/10
180849/180849 [==============================] - 0s - loss: 0.9667 - val_loss: 0.9410
Epoch 9/10
180849/180849 [==============================] - 0s - loss: 0.9662 - val_loss: 0.9494
Epoch 10/10
180849/180849 [==============================] - 0s - loss: 0.9

As a comparison: Without extra variables we got a train/test CRPS of 1.06/1.02

### Hidden nodes hyper-parameter tuning

Let's see what the impact of the number of hidden nodes is.

In [129]:
def build_and_run_hidden_model(hidden_nodes):
    hidden_model = build_hidden_model(hidden_nodes, feature_size=features_train.shape[1])
    hidden_model.compile(optimizer=Adam(0.01), loss=crps_cost_function)
    hidden_model.fit(features_train, target_train, epochs=20, 
                  batch_size=1024, verbose=0,
                  validation_data=[features_test, target_test])
    print(hidden_model.evaluate(features_train, target_train, verbose=0),
          hidden_model.evaluate(features_test, target_test, verbose=0))

In [130]:
for hidden_nodes in [10, 25, 50, 100, 250, 1000]:
    print(hidden_nodes)
    build_and_run_hidden_model(hidden_nodes)

10
0.974514032233 0.940623703924
25
0.945970456422 0.941868895928
50
0.94803293894 0.945383845405
100
0.915761278749 0.948940331101
250
0.89342511785 0.9350900614
1000
0.849479069772 0.956297336513


So with a larger network we get serious overfitting. Again, I would think that adding regularization with a larger network would maybe give the best results. Something to explore later.

## Additional variables with the embedding model

### Build model and predict for 2016

In [131]:
hidden_nodes, emb_size, feature_size = 50, 5, features_train.shape[1]

In [132]:
emb_model = build_emb_model(hidden_nodes, emb_size, max_id, feature_size)
emb_model.compile(optimizer=Adam(0.01), loss=crps_cost_function)
emb_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_54 (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_18 (Embedding)         (None, 1, 5)          2685        input_54[0][0]                   
____________________________________________________________________________________________________
input_53 (InputLayer)            (None, 24)            0                                            
____________________________________________________________________________________________________
flatten_18 (Flatten)             (None, 5)             0           embedding_18[0][0]               
___________________________________________________________________________________________

In [133]:
emb_model.fit([features_train, id_array_train], target_train, epochs=10, 
              batch_size=1024, 
              validation_data=[[features_test, id_array_test], target_test])

Train on 180849 samples, validate on 181718 samples
Epoch 1/10
180849/180849 [==============================] - 0s - loss: 1.8184 - val_loss: 0.9653
Epoch 2/10
180849/180849 [==============================] - 0s - loss: 0.9415 - val_loss: 0.9113
Epoch 3/10
180849/180849 [==============================] - 0s - loss: 0.9057 - val_loss: 0.8758
Epoch 4/10
180849/180849 [==============================] - 0s - loss: 0.8804 - val_loss: 0.8722
Epoch 5/10
180849/180849 [==============================] - 0s - loss: 0.8692 - val_loss: 0.8627
Epoch 6/10
180849/180849 [==============================] - 0s - loss: 0.8636 - val_loss: 0.8644
Epoch 7/10
180849/180849 [==============================] - 0s - loss: 0.8600 - val_loss: 0.8746
Epoch 8/10
180849/180849 [==============================] - 0s - loss: 0.8555 - val_loss: 0.8627
Epoch 9/10
180849/180849 [==============================] - 0s - loss: 0.8520 - val_loss: 0.8828
Epoch 10/10
180849/180849 [==============================] - 1s - loss: 0.8